In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import optuna

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import PowerTransformer
# from feature_engine.encoding import OrdinalEncoder

import warnings
warnings.filterwarnings("ignore")

# !pip install feature_engine --quiet

In [ ]:
# train = pd.read_csv("/kaggle/input/30-days-of-ml/train.csv")
# test = pd.read_csv("/kaggle/input/30-days-of-ml/test.csv")
# submission = pd.read_csv("/kaggle/input/30-days-of-ml/sample_submission.csv")

# X = train.drop(columns=['id', 'target'], axis=1)
# y = train['target']

# df_test = test.drop(columns=['id'], axis=1)

# # Select categorical columns
# categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# # Select numerical columns
# numerical_cols = X.select_dtypes(include=['float64']).columns.tolist()

## Feature Engineering

For numeric columns, I've used power transform. For categorical column, I've used Frequency Encoding. Please refer to [this](https://feature-engine.readthedocs.io/en/latest/index.html) for more information on other ways to feature engineer your training variables 

In [ ]:
# # Rare label encoding
# from feature_engine.encoding import RareLabelEncoder

# rare_encoder = RareLabelEncoder(tol=0.041, n_categories=1, replace_with='Z')
# rare_encoder.fit(X_train[categorical_cols])

# X_train_rare = rare_encoder.transform(X_train[categorical_cols]).add_prefix('rare_')
# X_test_rare = rare_encoder.transform(X_test[categorical_cols]).add_prefix('rare_')

# from feature_engine.encoding import OrdinalEncoder

# ordinal_encoder = OrdinalEncoder(encoding_method='ordered')
# ordinal_encoder.fit(X_train_rare, y)

# X_train_ordinal = ordinal_encoder.transform(X_train_rare).add_prefix('ordinal_')
# X_test_ordinal = ordinal_encoder.transform(X_test_rare).add_prefix('ordinal_')

# # Categorical Encoding Frequency Encoding
# from feature_engine.encoding import CountFrequencyEncoder

# count_enc = CountFrequencyEncoder(encoding_method='frequency')
# count_enc.fit(X_train_rare)

# X_train_freq = count_enc.transform(X_train_rare).add_prefix('freq_')
# X_test_freq= count_enc.transform(X_test_rare).add_prefix('freq_')

# from sklearn.preprocessing import PowerTransformer

# scaler = PowerTransformer()
# scaler.fit(X_train[numerical_cols])

# X_train_power = scaler.transform(X_train[numerical_cols])
# X_test_power = scaler.transform(X_test[numerical_cols])

# X_train_power = pd.DataFrame(X_train_power, columns=numerical_cols).add_prefix('power_')
# X_test_power = pd.DataFrame(X_test_power, columns=numerical_cols).add_prefix('power_')

# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# scaler.fit(X_train[numerical_cols])

# X_train_minmax = scaler.transform(X_train[numerical_cols])
# X_test_minmax = scaler.transform(X_test[numerical_cols])

# X_train_minmax = pd.DataFrame(X_train_minmax, columns=numerical_cols).add_prefix('minmax_')
# X_test_minmax = pd.DataFrame(X_test_minmax, columns=numerical_cols).add_prefix('minmax_')

# X = pd.concat([X_train_freq, X_train_ordinal, 
#                X_train_power, X_train_minmax], axis=1)

# X_test = pd.concat([X_test_freq, X_test_ordinal, 
#                     X_test_power, X_test_minmax], axis=1)

In [ ]:
# ordinal_encoder = OrdinalEncoder(encoding_method='ordered')
# ordinal_encoder.fit(X_train[categorical_cols], y)

# X_train_ordinal = ordinal_encoder.transform(X_train[categorical_cols])
# X_test_ordinal = ordinal_encoder.transform(X_test[categorical_cols])

# X = pd.concat([X_train_ordinal, X_train[numerical_cols]], axis=1)

# X_test = pd.concat([X_test_ordinal, X_test[numerical_cols]], axis=1)

# Modeling

## XGBoost

In [ ]:
# params = {
#     'random_state': 42,
#     'tree_method':'gpu_hist',  # Use GPU acceleration
#     'gamma': trial.suggest_float(
#         'gamma', 0.0, 20.0
#     ),
#     'reg_lambda': trial.suggest_float(
#         'reg_lambda', 0.0, 100
#     ),
#     'reg_alpha': trial.suggest_float(
#         'reg_alpha', 0.0, 100.0
#     ),
#     'learning_rate': trial.suggest_float(
#         'learning_rate', 0.2, 0.4
#     ),
#     'n_estimators': trial.suggest_int(
#         "n_estimators", 5000, 10000
#     ),
#     'max_depth': trial.suggest_int(
#         "max_depth", 3, 10
#     ),
#     'min_child_weight': trial.suggest_int(
#         'min_child_weight', 2, 20
#     ),
#     'subsample': trial.suggest_float(
#         'subsample', 0.5, 0.9
#     ),
#     'colsample_bytree': trial.suggest_float(
#         'colsample_bytree', 0.5, 0.9
#     ),
#     'colsample_bylevel': trial.suggest_float(
#         'colsample_bylevel', 0.5, 0.9
#     ),
#     'colsample_bynode': trial.suggest_float(
#         'colsample_bynode', 0.5, 0.9
#     ),
# }

In [ ]:
# # Uncomment this
# def objective(trial):
#     splits = 10
#     kfold = KFold(splits, shuffle=True, random_state=42)
    
#     # To select which parameters to optimize, please look at the XGBoost documentation:
#     # https://xgboost.readthedocs.io/en/latest/parameter.html
#     params = {
#         'random_state': 42,
#         'tree_method':'gpu_hist',  # Use GPU acceleration
#         'gamma': trial.suggest_float(
#             'gamma', 0.0, 0.2
#         ),
#         'reg_lambda': trial.suggest_float(
#             'reg_lambda', 50.0, 70.0
#         ),
#         'reg_alpha': trial.suggest_float(
#             'reg_alpha', 60.0, 80.0
#         ),
#         'learning_rate': trial.suggest_float(
#             'learning_rate', 0.2, 0.3
#         ),
#         'n_estimators': trial.suggest_int(
#             "n_estimators", 7000, 10000
#         ),
#         'max_depth': trial.suggest_int(
#             "max_depth", 3, 5
#         ),
#         'min_child_weight': trial.suggest_int(
#             'min_child_weight', 50, 100
#         ),
#         'subsample': trial.suggest_float(
#             'subsample', 0.9, 1.0
#         ),
#         'colsample_bytree': trial.suggest_float(
#             'colsample_bytree', 0.0, 0.1
#         ),
#         'colsample_bylevel': trial.suggest_float(
#             'colsample_bylevel', 0.7, 0.9
#         ),
#         'colsample_bynode': trial.suggest_float(
#             'colsample_bynode', 0.7, 0.9
#         )
#     }
    
#     # Creating an array of zeros for storing "out of fold" predictions
#     final_predictions = []
#     scores = []
    
#     # Generating folds and making training and prediction for each of 10 folds
#     for num, (train_idx, valid_idx) in enumerate(kfold.split(X)):
#         X_train, X_valid = X.loc[train_idx], X.loc[valid_idx]
#         y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]
#         X_test = df_test.copy()
        
#         ordinal_encoder = OrdinalEncoder()
#         X_train[categorical_cols] = ordinal_encoder.fit_transform(X_train[categorical_cols])
#         X_valid[categorical_cols] = ordinal_encoder.transform(X_valid[categorical_cols])
#         X_test[categorical_cols] = ordinal_encoder.transform(X_test[categorical_cols])

#         scaler = PowerTransformer()
#         X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
#         X_valid[numerical_cols] = scaler.transform(X_valid[numerical_cols])
#         X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])
        
#         model = XGBRegressor(**params)
#         model.fit(X_train, y_train,
#                   verbose=False,
#                   # These three parameters will stop training before a model starts overfitting 
#                   eval_set=[(X_train, y_train), (X_valid, y_valid)],
#                   eval_metric="rmse",
#                   early_stopping_rounds=300,
#                   )

#         # Getting mean test data predictions (i.e. divided by number of splits)
#         preds_test = model.predict(X_test)
#         preds_valid = model.predict(X_valid)
#         final_predictions.append(preds_test)

#         # Getting score for a fold model
#         rmse = mean_squared_error(y_valid, preds_valid, squared=False)
# #         print(f'VotingRegressor: Fold {num+1}, RMSE: {rmse}')
#         scores.append(rmse)

#     return np.mean(scores)

# study = optuna.create_study(direction='minimize', study_name='xgb_parameter_opt')
# study.optimize(objective, n_trials=2)
# print('Number of finished trials:', len(study.trials))
# print('Best trial:', study.best_trial.params)

# xgb_params = study.best_trial.params
# xgb_params # 0.7177683244544724

## LightGBM

In [ ]:
# def objective(trial, data=X, target=y):
#     splits = 10
#     kfold = KFold(splits, shuffle=True, random_state=42)
    
#     params = {
#         'device_type':'gpu',  # Use GPU acceleration
#         'num_leaves': trial.suggest_int(
#             'num_leaves', 40, 50
#         ),
#         'learning_rate': trial.suggest_float(
#             'learning_rate', 0.0, 0.2
#         ),
#         'n_estimators': trial.suggest_int(
#             "n_estimators", 5000, 10000
#         ),
#         'max_depth': trial.suggest_int(
#             'max_depth', 1, 10
#         ),
#         'reg_alpha': trial.suggest_float(
#             'reg_alpha', 20.0, 30.0
#         ),
#         'seed': 42,
#         'min_child_weight': trial.suggest_loguniform(
#             'min_child_weight', 1e-1, 1
#         ),
#     }
    
#     # Creating an array of zeros for storing "out of fold" predictions
#     final_predictions = []
#     scores = []
    
#     # Generating folds and making training and prediction for each of 10 folds
#     for num, (train_idx, valid_idx) in enumerate(kfold.split(X)):
#         X_train, X_valid = X.loc[train_idx], X.loc[valid_idx]
#         y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]

#         model = LGBMRegressor(**params)
#         model.fit(X_train, y_train,
#                   verbose=False,
#                   # These three parameters will stop training before a model starts overfitting 
#                   eval_set=[(X_train, y_train), (X_valid, y_valid)],
#                   eval_metric="rmse",
#                   early_stopping_rounds=300,
#                   )

#         # Getting mean test data predictions (i.e. divided by number of splits)
#         preds_test = model.predict(X_test)
#         preds_valid = model.predict(X_valid)
#         final_predictions.append(preds_test)

#         # Getting score for a fold model
#         rmse = mean_squared_error(y_valid, preds_valid, squared=False)
# #         print(f'VotingRegressor: Fold {num+1}, RMSE: {rmse}')
#         scores.append(rmse)

#     return np.mean(scores)

# study = optuna.create_study(direction='minimize', study_name='lgbm_parameter_opt')

# study.optimize(objective, n_trials=2)
# print('Number of finished trials:', len(study.trials))
# print('Best trial:', study.best_trial.params)

# lgbm_params = study.best_trial.params
# lgbm_params # 0.7181305083440424

## CatBoost

In [ ]:
# def objective(trial, data=X, target=y):
#     splits = 10
#     kfold = KFold(splits, shuffle=True, random_state=42)
    
#     # To select which parameters to optimize, please look at the XGBoost documentation:
#     # https://xgboost.readthedocs.io/en/latest/parameter.html
#     params = {
#         'task_type':'GPU',  # Use GPU acceleration
#         'learning_rate': trial.suggest_float(
#             'learning_rate', 0.0, 1.0
#         ),
#         'random_strength': trial.suggest_loguniform(
#             'random_strength', 1e-9, 10
#         ),
#         'bagging_temperature':trial.suggest_float(
#             'bagging_temperature', 0.0, 1.0
#         ),
#         'n_estimators': trial.suggest_int(
#             "n_estimators", 1000, 7000
#         ),
#         'depth': trial.suggest_int(
#             'max_depth', 1, 15
#         ),
#         'reg_lambda': trial.suggest_int(
#             'reg_alpha', 0.0, 50
#         ),
#         'random_state': 42,
# #         'silent': True
#         'verbose': 0
#     }
    
#     # Creating an array of zeros for storing "out of fold" predictions
#     final_predictions = []
#     scores = []
    
#     # Generating folds and making training and prediction for each of 10 folds
#     for num, (train_idx, valid_idx) in enumerate(kfold.split(X)):
#         X_train, X_valid = X.loc[train_idx], X.loc[valid_idx]
#         y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]

#         model = CatBoostRegressor(**params)
#         model.fit(X_train, y_train,
#                   verbose=False,
#                   # These three parameters will stop training before a model starts overfitting 
#                   eval_set=(X_valid, y_valid),
# #                   cat_features=categorical_cols,
#                   early_stopping_rounds=300,
#                   )

#         # Getting mean test data predictions (i.e. divided by number of splits)
#         preds_test = model.predict(X_test)
#         preds_valid = model.predict(X_valid)
#         final_predictions.append(preds_test)

#         # Getting score for a fold model
#         rmse = mean_squared_error(y_valid, preds_valid, squared=False)
# #         print(f'VotingRegressor: Fold {num+1}, RMSE: {rmse}')
#         scores.append(rmse)

#     return np.mean(scores)

# study = optuna.create_study(direction='minimize', study_name='cb_parameter_opt')
# study.optimize(objective, n_trials=30)
# print('Number of finished trials:', len(study.trials))
# print('Best trial:', study.best_trial.params)

# cb_params = study.best_trial.params
# cb_params

In [ ]:
lgbm_params = {
#     'device_type':'gpu',
#              'gpu_platform_id': 0,
#              'gpu_device_id': 0,
             'seed': 42,
             'num_leaves': 47,
             'learning_rate': 0.09369959637621708,
             'n_estimators': 5135,
             'max_depth': 2,
             'reg_alpha': 26.928438150963935,
             'min_child_weight': 0.37004853205139515
            }

xgb_params = {
#     'tree_method': 'gpu_hist',
#             'gpu_id':0,
#             'predictor':'gpu_predictor',
             'seed':42,
             'lambda': 64.68740340138797,
             'alpha': 40.27766966065833,
             'colsample_bytree': 0.031521737831702715,
             'subsample': 0.9970842659217856,
             'learning_rate': 0.34087533404234777,
             'n_estimators': 8390,
             'max_depth': 6,
             'min_child_weight': 112
            }

cb_params = {
#     'task_type':'GPU',
                  'random_state':42,
                  'learning_rate': 0.8058733338962497,
                 'n_estimators': 4536,
                 'max_depth': 1,
                 'reg_lambda': 6.82720069348649
                }

In [ ]:
xgb = XGBRegressor(**xgb_params)
# xgb.fit(X, y)

lgbm = LGBMRegressor(**lgbm_params)
# lgbm.fit(X, y)

cb = CatBoostRegressor(**cb_params, silent=True)
# cb.fit(X, y, logging_level='Silent')

vote_regressor = VotingRegressor([('xgboost', xgb), ('lightGBM', lgbm), ('catboost',cb)], 
                        weights=[0.6,0.35,0.05])

In [ ]:
train = pd.read_csv("/kaggle/input/30-days-of-ml/train.csv")
test = pd.read_csv("/kaggle/input/30-days-of-ml/test.csv")
submission = pd.read_csv("/kaggle/input/30-days-of-ml/sample_submission.csv")

X = train.drop(columns=['id', 'target'], axis=1)
y = train['target']

df_test = test.drop(columns=['id'], axis=1)

# Select categorical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# Select numerical columns
numerical_cols = X.select_dtypes(include=['float64']).columns.tolist()

In [ ]:
%%time
splits = 10
kfold = KFold(splits, shuffle=True, random_state=42)

# Creating an array of zeros for storing "out of fold" predictions
final_predictions = []
scores = []
    
# Generating folds and making training and prediction for each of 10 folds
for num, (train_idx, valid_idx) in enumerate(kfold.split(X)):
    X_train, X_valid = X.loc[train_idx], X.loc[valid_idx]
    y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]
    X_test = df_test.copy()

    ordinal_encoder = OrdinalEncoder()
    X_train[categorical_cols] = ordinal_encoder.fit_transform(X_train[categorical_cols])
    X_valid[categorical_cols] = ordinal_encoder.transform(X_valid[categorical_cols])
    X_test[categorical_cols] = ordinal_encoder.transform(X_test[categorical_cols])

    scaler = PowerTransformer()
    X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
    X_valid[numerical_cols] = scaler.transform(X_valid[numerical_cols])
    X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

    model = VotingRegressor([('xgboost', xgb), 
                             ('lightGBM', lgbm), 
                             ('voting_regression',vote_regressor)], 
                            weights=[0.2,0.1,0.7])

#     model = StackingRegressor([('xgboost', xgb), ('lightGBM', lgbm), ('catboost',cb)])
    
    model.fit(X_train, y_train)

    # Getting mean test data predictions (i.e. divided by number of splits)
    preds_test = model.predict(X_test)
    preds_valid = model.predict(X_valid)
    final_predictions.append(preds_test)
    
    # Getting score for a fold model
    rmse = mean_squared_error(y_valid, preds_valid, squared=False)
#     print(f'VotingRegressor: Fold {num+1}, RMSE: {rmse}')
    scores.append(rmse)
    
print(np.mean(scores), np.std(scores))

In [ ]:
%%time
vote_regressor = VotingRegressor([('xgboost', xgb), ('lightGBM', lgbm), ('catboost',cb)], 
                        weights=[0.6,0.35,0.05])

cv_results = cross_val_score(vote_regressor, X, y, cv=5, scoring='neg_mean_squared_error')
np.sqrt(np.abs(cv_results.mean()))

In [ ]:
%%time
vote_regressor2 = VotingRegressor([('xgboost', xgb), 
                                  ('lightGBM', lgbm), 
                                  ('voting_regression',vote_regressor)], 
                        weights=[0.2,0.1,0.7])

cv_results = cross_val_score(vote_regressor2, X, y, cv=5, scoring='neg_mean_squared_error')
np.sqrt(np.abs(cv_results.mean()))

In [ ]:
y_pred = np.mean(np.column_stack(final_predictions), axis=1)

submission['target'] = y_pred

# Save submission_df to csv and upload to Kaggle
submission.to_csv(r'submission_oof_stackregressor.csv', index=False)

In [ ]:
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.preprocessing import OrdinalEncoder, PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [ ]:
# df_train = pd.read_csv(r'../input/30-days-of-ml/train.csv')
# kf = model_selection.KFold(n_splits=10, shuffle=True, random_state=42)

# df_train['kfold'] = -1

# for fold, (train_idx, valid_idx) in enumerate(kf.split(X=df_train)):
#     df_train.loc[valid_idx, 'kfold'] = fold

# df_train.to_csv('train_folds.csv', index=False)

In [ ]:
df = pd.read_csv('train_folds.csv')
df_test = pd.read_csv('../input/30-days-of-ml/test.csv')
submission = pd.read_csv('../input/30-days-of-ml/sample_submission.csv')

In [ ]:
useful_features = [c for c in df.columns if c not in ['id', 'target', 'kfold']]
df_test = df_test[useful_features]

# Select categorical columns
categorical_cols = df[useful_features].select_dtypes(include=['object']).columns.tolist()

# Select numerical columns
numerical_cols = df[useful_features].select_dtypes(include=['float64']).columns.tolist()

In [ ]:
final_test_predictions = []
final_valid_predictions = {}
scores = []

for fold in range(10):
    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    valid_ids = xvalid.id.values.tolist()
    
    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = OrdinalEncoder()
    xtrain[categorical_cols] = ordinal_encoder.fit_transform(xtrain[categorical_cols])
    xvalid[categorical_cols] = ordinal_encoder.transform(xvalid[categorical_cols])
    xtest[categorical_cols] = ordinal_encoder.transform(xtest[categorical_cols])

    scaler = PowerTransformer()
    xtrain[numerical_cols] = scaler.fit_transform(xtrain[numerical_cols])
    xvalid[numerical_cols] = scaler.transform(xvalid[numerical_cols])
    xtest[numerical_cols] = scaler.transform(xtest[numerical_cols])
    
    xgb_params = {
#             'tree_method': 'gpu_hist',
#             'gpu_id':0,
#             'predictor':'gpu_predictor',
             'seed':42,
             'lambda': 64.68740340138797,
             'alpha': 40.27766966065833,
             'colsample_bytree': 0.031521737831702715,
             'subsample': 0.9970842659217856,
             'learning_rate': 0.34087533404234777,
             'n_estimators': 8390,
             'max_depth': 6,
             'min_child_weight': 112
            }
    
    # Define the model 
    model = XGBRegressor(random_state=fold, n_jobs=4, **xgb_params)

    # Train the model (will take about 10 minutes to run)
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)

    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    scores.append(rmse)

print(np.mean(scores), np.std(scores)) # 0.7177641712350958 0.002237195974190003
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_1"]
final_valid_predictions.to_csv("train_pred_1.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_1"]
sample_submission.to_csv("test_pred_1.csv", index=False)

In [ ]:
preds = np.mean(np.column_stack(final_predictions), axis=1)
preds

submission['target'] = preds
submission.to_csv('submission.csv', index=False)